In [3]:
# Indução de regras
"""
    Entendendo como que o aprendizado de regras funciona:
    
    Quando pegamos uma arvore de regras, o algoritmo vai fazer caminhos até obter aquele valor, ja no caso do do naïve bayes, ele vai fazer uma verificação estatistica das condições
    e vai selecionar a mais provavel, de acordo com os levantamentos feitos da base de dados
    
    Ja na apprendizagem por regras, o algoritmo vai criar regras fixas para obter as melhores condições.
    
"""
import pickle
import pandas as pd

with open('./zegodata.pkl', "rb") as data:
    previsores, classe = pickle.load(data)

with open('./zegotestdata.pkl', 'rb') as data:
    previsores_teste, classe_teste = pickle.load(data)


# Testando valores e verificando o tamanho das bases coletadas e mantidas no pickle

# print(previsores, classe)
# print(previsores_teste, classe_teste)
# print(len(previsores), len(previsores_teste))

# Algoritmo OpenR(ules)
<h3>Neste tipo a gente precisa testar antes com uma quantidade menor de dados para testar as regras que queremos aplicar</h3>


Explicando e exemplificando a história

nos fazemos uma tabela, onde a primeira coluna é o atributo, a segunda coluna é a classe que queremos verificar, a terceira coluna é a regra, a quarta coluna é o erro e a quinta é o erro somado, então:


<!-- Lembrar de fazer uma tabela de verdade aqui, nao deixar simplesmente alinhado pq nao vai adiantar muito -->
|Atributo     |quantidade         |regras                             |erro      |total erro|
|--- |--- |--- |--- |--- |
|A            |ruim = 10          |se ruim, entao alto                 |1/5         |       |
|b            |desconhecida = 3   |se desconhecida entao moderado      |3/5         |6/15   |
|c            |boa = 1            |se boa entao baixo                  |1/5         |        |

# Algoritmo PRISM

No caso do prism, nos pegamos apenas uma condição, e dela nos vamos pegar a maior abrangencia e determinar qual vai ser a melhor condiaçao para o nosso algoritmo




In [4]:
# Como o sklearn nao tem este tipo de aprendizagem, nos iremos utilizar a lib orange3
import Orange

base_AR = Orange.data.Table("AR_Data.csv")
base_AR

[[CC, <50%, <50%, <200 | One-Time],
 [CC, <50%, <50%, <200 | One-Time],
 [ACH, <50%, <50%, <200 | Recurring],
 [ACH, <50%, <50%, <200 | Recurring],
 [CC, <50%, <50%, <200 | One-Time],
 ...
]

In [5]:
base_AR[0].domain

[PaymentType, Community Range, Door Range, Payment range | recurrence]

In [6]:
# Classificando os nossos dados

cn2 = Orange.classification.rules.CN2Learner()
regras = cn2(base_AR)

# E Aqui geramos a lista de quais regras o Orange vai criar
for r in regras.rule_list:
    print(r)

IF Community Range==<=20% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Community Range==<70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Community Range!=>70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Door Range==<70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Door Range==>70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Payment range!=<200 THEN recurrence=One-Time 
IF Door Range==>70% AND Community Range!=>70% THEN recurrence=One-Time 
IF Door Range==<70% AND Community Range==<70% THEN recurrence=One-Time 
IF Door Range!=<=20% AND Community Range==<70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Door Range!=<50% THEN recurrence=One-Time 
IF PaymentType!=ACH THEN recurrence=One-Time 
IF Door Range==>70% AND Payment range==<1000 THEN recurrence=Recurring 
IF Community Range==<50% AND Payment range==>1000 AND Door Range==<70% THEN recurrence=One-Time 
IF Door Range==>70% AND Payment range!=>1000 THEN recurrence=Recurring 
IF Community Range==<50% AND 

In [7]:
# Dividindo a base e verificando os tamanhos

treino_ar = Orange.evaluation.testing.sample(base_AR, n = 0.6)
teste_ar = treino_ar[1]
treino_ar = treino_ar[0]

print(len(treino_ar), len(teste_ar))

300000 200000


In [8]:
regras = cn2(treino_ar)

for regra in regras.rule_list:
    print(regra)

IF Community Range==<=20% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Community Range==<70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Community Range!=>70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Door Range==<70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Door Range==>70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Payment range!=<200 THEN recurrence=One-Time 
IF Door Range==>70% AND Community Range!=>70% THEN recurrence=One-Time 
IF Door Range==<70% AND Community Range==<70% THEN recurrence=One-Time 
IF Door Range!=<=20% AND Community Range==<70% THEN recurrence=One-Time 
IF PaymentType!=ACH AND Door Range!=<50% THEN recurrence=One-Time 
IF PaymentType!=ACH THEN recurrence=One-Time 
IF Door Range==>70% AND Payment range!=>1000 AND Payment range!=<1000 THEN recurrence=Recurring 
IF Community Range==<50% AND Payment range==>1000 AND Door Range==<70% THEN recurrence=One-Time 
IF Door Range==>70% AND Payment range!=>1000 THEN recurrence=Recurring 
IF C

In [9]:
# Aqui estamos ja dando tanto a base de teste, tanto a base de treino, e para testar os valores, nos precisamos inserir uma função lambda para inserir as regras

previsoes = Orange.evaluation.TestOnTestData(treino_ar, teste_ar, [lambda testdata: regras])


Orange.evaluation.CA(previsoes)

# Neste caso, tivemos uma previsão correta de 84%

array([0.84388])